In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from transformers import TFBertModel,  BertConfig, BertTokenizerFast


ModuleNotFoundError: No module named 'transformers'

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
model_name = 'bert-base-multilingual-cased'
max_length = 200

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [4]:
train = pd.read_csv('../dataset/train.csv')
valid = pd.read_csv('../dataset/val.csv')
test = pd.read_csv('../dataset/test.csv')

In [5]:
train

,Emotion,Comment
0,0,On days when I feel close to my partner and ot...
1,1,Every time I imagine that someone I love or I ...
2,2,When I had been obviously unjustly treated and...
3,3,When I think about the short time that we live...
4,4,At a gathering I found myself involuntarily si...
...,...,...
5179,1,My parents were out and I was the eldest at ho...
5180,5,Two years back someone invited me to be the tu...
5181,5,I had taken the responsibility to do something...
5182,1,I was at home and I heard a loud sound of spit...


In [6]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [7]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Comment.values, train.Emotion.values, 
                                                  stratify=train.Emotion.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

train['Comment'].apply(lambda x:len(str(x).split())).max()

168

In [8]:
yvalid.shape

(1037,)

In [16]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index


In [10]:
xvalid_pad.shape

(1037, 168)

In [17]:
def build_model(transformer):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    sequence_output = transformer(input_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(7, activation='softmax')(cls_token)

    model = Model(inputs=input_ids, outputs=out)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(Adam(lr=1e-5), loss=loss, metrics=['accuracy'])

    return model

In [11]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,300,input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 168, 300)          2294100   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 7)                 707       
Total params: 2,334,907
Trainable params: 2,334,907
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(xtrain_pad, ytrain,epochs=10, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/10
65/65 [==============================] - 6s 85ms/step - loss: 1.9406 - accuracy: 0.1717
Epoch 2/10
65/65 [==============================] - 6s 85ms/step - loss: 1.7842 - accuracy: 0.5348
Epoch 3/10
65/65 [==============================] - 6s 92ms/step - loss: 1.4467 - accuracy: 0.8182
Epoch 4/10
65/65 [==============================] - 6s 92ms/step - loss: 1.2727 - accuracy: 0.9298
Epoch 5/10
65/65 [==============================] - 6s 91ms/step - loss: 1.2189 - accuracy: 0.9648
Epoch 6/10
65/65 [==============================] - 6s 86ms/step - loss: 1.1939 - accuracy: 0.9812
Epoch 7/10
65/65 [==============================] - 6s 86ms/step - loss: 1.1837 - accuracy: 0.9870
Epoch 8/10
65/65 [==============================] - 6s 86ms/step - loss: 1.1785 - accuracy: 0.9899
Epoch 9/10
65/65 [==============================] - 6s 85ms/step - loss: 1.1760 - accuracy: 0.9916
Epoch 10/10
65/65 [==============================] - 6s 85ms/step - loss: 1.1740 - accuracy: 0.9930


In [14]:
model.evaluate(xvalid_pad, yvalid)

33/33 [==============================] - 0s 15ms/step - loss: 1.8359 - accuracy: 0.3105


[1.8359228372573853, 0.3105110824108124]